# Notebook 05: Data merging and control variable construction

In [1]:
import pandas as pd
import numpy as np

load data

In [5]:
file_path = 'data/instrument_panel.csv'
df_panel = pd.read_csv(file_path)
df_panel['Month_ID'] = pd.PeriodIndex(df_panel['Month_ID'], freq='M')

In [6]:
file_path = 'data/final_broker_decoder_ring.csv'
df_shocks = pd.read_csv(file_path)

In [7]:
# keep relevant columns for merge
shock_map = df_shocks[['Identified_ID', 'Shock_Date', 'Broker_Name']].copy()
shock_map.rename(columns={'Identified_ID': 'Event_ID'}, inplace=True)

# merge shock date into panel
df_panel = pd.merge(df_panel, shock_map, on=['Event_ID', 'Broker_Name'], how='left')
df_panel['Shock_Date'] = pd.to_datetime(df_panel['Shock_Date'])

In [8]:
file_path = 'data/monthly_outcomes.csv'
df_outcomes = pd.read_csv(file_path)
df_outcomes['Month_ID'] = pd.PeriodIndex(df_outcomes['Month_ID'], freq='M')

In [9]:
# merge
df_merge_1 = pd.merge(
    df_panel,
    df_outcomes,
    on=['CUSIP', 'Month_ID'],
    how='left'
)
print(f"Merged outcomes shape: {df_merge_1.shape}")

Merged outcomes shape: (70344, 18)


In [10]:
file_path = 'data/compustat_annual.csv'
cols_comp = [
    'gvkey', 'tic', 'cusip', 'fyear', 'datadate',
    'at',      # Total Assets
    'ni',      # Net Income
    'dltt',    # Long Term Debt
    'dlc',     # Debt in Current Liab
    'prcc_f',  # Price Close (Fiscal)
    'csho',    # Shares Outstanding
    'ceq',     # Common Equity
    'xrd',     # R&D Expense
    'sich'     # Historical SIC Code (Industry)
]
df_fund = pd.read_csv(file_path, usecols=lambda x: x.lower() in cols_comp)
df_fund.columns = df_fund.columns.str.lower()

Construct control variables

In [11]:
# firm size = log(Total Assets)
df_fund['Size'] = np.log(df_fund['at'].replace(0, np.nan))

# return on assets = Net Income / Total Assets
df_fund['ROA'] = df_fund['ni'] / df_fund['at']

# leverage = (Long Term Debt + Current Debt) / Total Assets
df_fund['Total_Debt'] = df_fund['dltt'].fillna(0) + df_fund['dlc'].fillna(0)
df_fund['Leverage'] = df_fund['Total_Debt'] / df_fund['at']

# market-to-book = (Price * Shares) / Common Equity
df_fund['Mkt_Cap'] = df_fund['prcc_f'] * df_fund['csho']
df_fund['MTB'] = df_fund['Mkt_Cap'] / df_fund['ceq']

# r&d intensity = R&D Expense / Total Assets
df_fund['RnD_Exp'] = df_fund['xrd'].fillna(0)
df_fund['Opaqueness'] = df_fund['RnD_Exp'] / df_fund['at']

# industry fixed effects
# we use the first 2 digits of the SIC code (major group)
df_fund['Industry_SIC2'] = df_fund['sich'].astype(str).str[:2]

Lagging and merging strategy

Stock returns in **2015** should be predicted by financial data from **2014**.

In [12]:
df_fund['Join_Year'] = df_fund['fyear'] + 1
df_fund['CUSIP_8'] = df_fund['cusip'].astype(str).str[:8]

# de-duplicate compustat
df_fund = df_fund.sort_values(['CUSIP_8', 'Join_Year', 'datadate']) \
    .drop_duplicates(subset=['CUSIP_8', 'Join_Year'], keep='last')

# select final controls
controls_to_keep = ['CUSIP_8', 'Join_Year', 'Size', 'ROA', 'Leverage', 'MTB', 'Opaqueness', 'Industry_SIC2']
df_fund_clean = df_fund[controls_to_keep].copy()

final merge

In [13]:
df_merge_1['Join_Year'] = df_merge_1['Month_ID'].dt.year
df_final = pd.merge(
    df_merge_1,
    df_fund_clean,
    left_on=['CUSIP', 'Join_Year'],
    right_on=['CUSIP_8', 'Join_Year'],
    how='left'
)

# drop redundant merge columns
df_final.drop(columns=['CUSIP_8'], inplace=True)

In [14]:
output_path = 'data/final_regression_panel.csv'
df_final.to_csv(output_path, index=False)

In [15]:
print(f"Total Observations: {len(df_final)}")
print(f"Columns: {df_final.columns.tolist()}")

Total Observations: 70344
Columns: ['CUSIP', 'Coverage', 'Event_Month', 'Event_ID', 'Broker_Name', 'Treated', 'Post', 'Treated_Post', 'Month_ID', 'Shock_Date', 'TICKER', 'NCSKEW', 'DUVOL', 'Volatility', 'Avg_Spread', 'Avg_Amihud', 'Avg_Turnover', 'Price_Delay', 'Join_Year', 'Size', 'ROA', 'Leverage', 'MTB', 'Opaqueness', 'Industry_SIC2']
